In [129]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

groq_api_key= os.getenv("GROQ_API_KEY")
llm= ChatGroq(model_name= "Mixtral-8x7b-32768", groq_api_key= groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020B4E238440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020B4E23B320>, model_name='Mixtral-8x7b-32768', groq_api_key=SecretStr('**********'))

In [130]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [131]:
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= ChatMessageHistory()
    return store[session_id]

In [132]:
import random
import string

def random_session_id(length:int):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

session_id= random_session_id(5)

In [133]:
config={"configurable":{"session_id":f"{session_id}"}}

In [228]:
system_prompt= """ 
ROLE: You are an expert Python software developer specializes in Python coding.
TASK: Your task is to generate dynamic python code exclusively for the prompt provided.
Ensure that the code is properly intended and follows best practices for readability and maintainability.
Include function definitions where possible and ensure code is error free.

INSTRUCTIONS:
1. Ensure that your responses are strictly dynamic python code enclosed in ``` only, with no textual information.
Make sure you only return the code and nothing else.
2. If user provides specific input values(e.g., path, filenames), Replace the example usage variable with user's input.
(This replacement is mandatory where user input is provided)
3. When fixing or explaining code, use comments to describe changes or provide explanations.
4. Ensure that the code doesn't have any indentation errors.
5. Specify the input and output data type in the function definition mandatory.

ADDITIONALS:
1. Once the function is generated, at the end write only 1 example using the generated function.
2. It should follow this pattern:
Example: 
result= my_function()


Example of proper variable usage and user input replacement:
```python
def is_palindrome(text):
    if text==text[::-1]:
        return True
    else:
        return False

#Example usage with variables:
input_string= 'malyalam'
result= is_palindrome(input_string)
"""

In [229]:
from langchain_core.prompts import MessagesPlaceholder
prompt= ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder(variable_name="user_message")
    ]
)

print(prompt)

input_variables=['user_message'] input_types={'user_message': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=" \nROLE: You are an expert Python software developer specializes in Python coding.\nTASK: Your task is to generate dynamic python code exclusively for the prompt provided.\nEnsure that the code is properly intended and follows best practices for readability and maintainability.\nInclude function definitions where possible and ensure code is error free.\n\nINSTRUCTIONS:\n1. Ensure that your responses are strictly dynamic python code enclosed in ``` only, with no textual information.\nMake sure you only return the code and nothing else.\n2. If user provid

In [230]:
chain= prompt|llm
with_message_history= RunnableWithMessageHistory(chain,get_session_history)

In [231]:
def get_llm_response(user_prompt, with_message_history, config):
    response= with_message_history.invoke(
        {'user_message': [HumanMessage(content=user_prompt)]},
        config=config
    )
    return response.content

In [232]:
user_prompt= "python function to return square of a given number"
response= get_llm_response(user_prompt, with_message_history,config)

print(response)

Here's the Python code to return the square of a given number:

```python
def square(number: int) -> int:
    """
    This function returns the square of a given number.

    :param number: The number for which square is to be found.
    :return: The square of the given number.
    """
    return number ** 2
```
Example:
--------

```python
result = square(5)
```


In [250]:
store={}

In [251]:
user_prompt= "python function to find area of a rectangle"
response= get_llm_response(user_prompt,with_message_history,config)

print(response)

Here is the dynamic Python code to find the area of a rectangle:

```python
def calculate_rectangle_area(length: float, width: float) -> float:
    """
    This function calculates the area of a rectangle.

    :param length: Length of the rectangle.
    :param width: Width of the rectangle.
    :return: Area of the rectangle.
    """
    area = length * width
    return area

# Example usage with variables:
length = 5.0
width = 3.0
result = calculate_rectangle_area(length, width)
```
Example usage:
-----------------
result = calculate\_rectangle\_area(5.0, 3.0)


In [252]:
keys= list(store.keys())
first_key= keys[0]
first_key

'1S78Y'

In [253]:
for i in store[first_key].messages:
    print(i)

content='python function to find area of a rectangle'
content='Here is the dynamic Python code to find the area of a rectangle:\n\n```python\ndef calculate_rectangle_area(length: float, width: float) -> float:\n    """\n    This function calculates the area of a rectangle.\n\n    :param length: Length of the rectangle.\n    :param width: Width of the rectangle.\n    :return: Area of the rectangle.\n    """\n    area = length * width\n    return area\n\n# Example usage with variables:\nlength = 5.0\nwidth = 3.0\nresult = calculate_rectangle_area(length, width)\n```\nExample usage:\n-----------------\nresult = calculate\\_rectangle\\_area(5.0, 3.0)' response_metadata={'token_usage': {'completion_tokens': 168, 'prompt_tokens': 356, 'total_tokens': 524, 'completion_time': 0.269911644, 'prompt_time': 0.024911978, 'queue_time': 0.0007649519999999993, 'total_time': 0.294823622}, 'model_name': 'Mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': Non

In [254]:
import re
def clean_generated_code(response):
    code_blocks = re.findall(r'```python(.*?)```', response, re.DOTALL)
    extracted_code = '\n'.join(block.strip() for block in code_blocks)
    return extracted_code

clean_response= clean_generated_code(response)
print(clean_response)

def calculate_rectangle_area(length: float, width: float) -> float:
    """
    This function calculates the area of a rectangle.

    :param length: Length of the rectangle.
    :param width: Width of the rectangle.
    :return: Area of the rectangle.
    """
    area = length * width
    return area

# Example usage with variables:
length = 5.0
width = 3.0
result = calculate_rectangle_area(length, width)


In [255]:
def extract_missing_modules(error_msg):
    missing_modules= re.findall(r"ModuleNotFoundError: No module named '(\w+)'", error_msg)
    return missing_modules


def install_module(module_name):
    import subprocess
    result= subprocess.run(["pip", "install", module_name], check=True)
    if result.returncode==0:
        print(f"Module '{module_name}' installed successfully")
    else:
        print(f"Failed to install module '{module_name}'")

    return result.returncode==0

In [256]:
import subprocess
import sys

def run_code(code):
    try:
        result=  subprocess.run([f"{sys.executable}","-c", code], capture_output=True, text=True, timeout=3000)
        if result.returncode!=0:
            missing_modules= extract_missing_modules(result.stderr)
            for module in missing_modules:
                install_module(module)
            result= subprocess.run([f"{sys.executable}","-c", code], capture_output=True, text=True, timeout=3000)
            if result.returncode!=0:
                return False, result.stderr
        else:
            return True, result.stdout
    except subprocess.TimeoutExpired:
        return False, "Execution timed out"
    except Exception as e:
        return False, str(e)

In [257]:
run_code(clean_response)

(True, '')

In [258]:
print(clean_response)

def calculate_rectangle_area(length: float, width: float) -> float:
    """
    This function calculates the area of a rectangle.

    :param length: Length of the rectangle.
    :param width: Width of the rectangle.
    :return: Area of the rectangle.
    """
    area = length * width
    return area

# Example usage with variables:
length = 5.0
width = 3.0
result = calculate_rectangle_area(length, width)


In [263]:
def retrieve_input_vars_and_type(code_string: string)->dict:
    input_params_pattern = r"def\s+\w+\(([^)]+)\)"
    input_type_pattern = r"(\w+):\s*(\w+)"

    # Extract input parameters and types
    input_params_match = re.search(input_params_pattern, code_string)
    if input_params_match:
        input_params = input_params_match.group(1).strip()
        input_vars = re.findall(input_type_pattern, input_params)
        
        input_dict = {var: type_ for var, type_ in input_vars}
        return input_dict

In [265]:
input_vars_dict= retrieve_input_vars_and_type(clean_response)
input_vars_dict

{'length': 'float', 'width': 'float'}

In [273]:
list(input_vars_dict.values())

['float', 'float']

In [152]:
exec(clean_response)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


In [153]:
fibonacci_series(5)

[0, 1, 1, 2, 3]

In [207]:
def extract_function(code_string):
    lines = code_string.splitlines()
    function_lines = []
    inside_function = False
    indentation_level = None

    for line in lines:
        # Check if the line starts the function definition
        if line.strip().startswith("def "):
            inside_function = True
            function_lines.append(line)
            indentation_level = len(line) - len(line.lstrip())
        elif inside_function:
            current_indentation = len(line) - len(line.lstrip())

            # If the line is less indented or empty, we've reached the end of the function
            if current_indentation <= indentation_level and line.strip() != "":
                break

            function_lines.append(line)

    return "\n".join(function_lines)

In [208]:
extracted_function = extract_function(clean_response)
print(extracted_function)

def square(number: int) -> int:
    """
    Function to return the square of a given number.

    :param number: The number to find the square of.
    :return: The square of the given number.
    """
    return number ** 2



In [183]:
def extract_function_name(code_string):
    # Use regex to find the function name
    match = re.search(r'def\s+(\w+)\s*\(', code_string)
    if match:
        return match.group(1)
    return None

In [184]:
function_name

'is_palindrome'

In [267]:
def call_function_by_name(function_name, *args, **kwargs):
    # Use globals() or locals() to call the function dynamically
    if function_name in globals():
        return globals()[function_name](*args, **kwargs)
    elif function_name in locals():
        return locals()[function_name](*args, **kwargs)
    else:
        raise NameError(f"Function '{function_name}' is not defined.")

In [188]:
function_name= extract_function_name(clean_response)
exec(extracted_function)

if function_name:
    result = call_function_by_name(function_name, 12345)  # Example: calling fibonacci_series(7)
    print(result)

False


In [269]:
user_input= '12.0 30.0'
inputs= tuple(user_input.split(' '))
inputs

('12.0', '30.0')

In [274]:
def parse_user_input(user_input, type_mapping):
    input_values = user_input.split(' ')
    
    # Ensure that the number of inputs matches the number of types
    if len(input_values) != len(type_mapping):
        raise ValueError("Number of input values does not match the number of expected types.")
    
    # Convert input values to the appropriate data types
    converted_values = []
    for value, dtype in zip(input_values, type_mapping):
        if dtype == 'float':
            converted_values.append(float(value))
        elif dtype == 'int':
            converted_values.append(int(value))
        elif dtype == 'str':
            converted_values.append(value)
        elif dtype == 'bool':
            converted_values.append(value.lower() in ['true', '1', 'yes'])
        else:
            raise ValueError(f"Unsupported data type: {dtype}")
    
    return tuple(converted_values)

# Example user input and type mapping
user_input = "12 kishan"
type_mapping = ['float', 'str']

# Parse and convert the user input
parsed_inputs = parse_user_input(user_input, type_mapping)
print(parsed_inputs)


(12.0, 'kishan')
